In [1]:
import torch
import numpy as np

In [2]:
import sys
from pathlib import Path

module_path = Path("cisco-time-series-model") / "1.0-preview"
parent = module_path.resolve()  # adjust to yours
# print(parent)  # make Python look into that folder for modules

if str(parent) not in sys.path:
	sys.path.insert(0, str(parent))

In [3]:
from modeling import CiscoTsmMR, TimesFmHparams, TimesFmCheckpoint

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded PyTorch TimesFM, likely because python version is 3.10.19 (main, Oct 21 2025, 16:43:05) [GCC 11.2.0].


In [4]:
rng = np.random.default_rng(42)

In [5]:
## Sample data
T = 512 * 60
hours = (T + 59) // 60
k = np.arange(hours, dtype=np.float32)
h = (80 + 0.1 * k) * (1 + 0.25 * np.sin(2 * np.pi * k / 24))
t = np.arange(T, dtype=np.float32)

In [6]:
input_series_1 = h[(t // 60).astype(int)] * (1 + 0.05 * np.sin(2 * np.pi * t / 30)) + rng.normal(0, 0.4, size=T)

In [7]:
"gpu" if torch.cuda.is_available() else "cpu"

'gpu'

In [8]:
# Hyperparameters
hparams = TimesFmHparams(
    num_layers=50,
    use_positional_embedding=False,
    backend="gpu" if torch.cuda.is_available() else "cpu",
)


In [18]:
# 2) Force HF into offline mode (no network calls)
os.environ["HF_HUB_ENABLE_OFFLINE"] = "1"


In [ ]:
model_dir = Path("cisco-ts-model-1.0-preview")
ckpt_file = model_dir / "torch_model.pt"   # adjust if your file has a different name

# Load model checkpoint
ckpt = TimesFmCheckpoint(
    path=ckpt_file,
    )

In [28]:
# Load model
model = CiscoTsmMR(
    hparams=hparams,
    checkpoint=ckpt,
    use_resolution_embeddings=True,
    use_special_token=True,
)

In [33]:
print(input_series_1)

[ 80.12188683  80.41564851  81.9271336  ... 158.18353734 159.84958834
 160.59672067]


In [30]:
# Model Inference
forecast_preds = model.forecast(input_series_1, horizon_len=128)

# Access forecast mean and quantiles of each series
mean_forecast = forecast_preds[0]['mean'] # (128,)
quantiles = forecast_preds[0]['quantiles'] # dict with keys as quantile levels (0.1, 0.2, ...., 0.9) and values as (128,) numpy arrays


In [31]:
print(mean_forecast)

[162.8717  164.54495 166.2567  167.41876 168.21674 168.94269 169.21669
 169.5842  169.27684 168.72615 168.04587 167.03725 165.82735 164.21326
 162.4862  161.1493  159.36902 158.04994 156.75578 155.62207 154.62755
 154.32658 153.89235 154.11415 154.30884 154.90488 155.5547  156.45207
 158.05225 159.45978 161.25491 162.63058 165.07884 166.53377 167.80432
 168.30075 168.7428  169.06967 169.02109 168.63986 167.8889  166.64
 165.48909 163.8709  161.79504 160.24774 158.53288 156.92635 155.31128
 154.27325 153.39563 152.91637 152.52338 152.68259 152.79149 153.57155
 154.3992  155.53224 156.85039 157.91435 160.34932 161.7961  163.4616
 164.51834 165.25581 165.95377 166.51898 166.48224 166.0221  165.71788
 164.64453 163.6025  162.2258  160.60214 158.65056 157.23282 155.63556
 154.2157  152.58195 151.49596 150.44301 150.12029 149.67503 149.56094
 149.8421  150.50168 151.33171 152.51913 154.20279 155.24576 156.91078
 158.17026 160.6151  161.9789  162.97043 163.57703 164.09271 164.14357
 164.28206

In [32]:
print(quantiles)

{'0.1': array([162.21042, 163.4449 , 165.17612, 166.28859, 167.18822, 167.8659 ,
       168.31197, 168.37825, 168.17   , 167.62685, 166.92735, 165.83377,
       164.59653, 163.0275 , 161.37398, 160.07164, 158.4052 , 156.85103,
       155.637  , 154.4707 , 153.48917, 153.08144, 152.74796, 152.89127,
       153.2216 , 153.71378, 154.33153, 155.19914, 156.80263, 158.13374,
       159.91122, 161.19594, 163.64224, 164.95187, 166.42427, 166.78674,
       167.32083, 167.58363, 167.61403, 167.20992, 166.43575, 165.15166,
       163.93478, 162.39574, 160.4742 , 158.90607, 157.1997 , 155.45312,
       153.88664, 152.78131, 152.02254, 151.44495, 151.12971, 151.23155,
       151.46265, 152.10724, 152.84935, 153.7203 , 155.00716, 156.02303,
       158.3745 , 159.74323, 161.53056, 162.37132, 163.15245, 163.60373,
       164.34277, 164.32407, 163.87868, 163.51263, 162.42195, 161.2878 ,
       160.01472, 158.42801, 156.55078, 155.02899, 153.43898, 151.98193,
       150.58998, 149.38484, 148.347  , 147

In [34]:
# You can also forecast multiple series at once
T = 25_000
hours = (T + 59) // 60
k = np.arange(hours, dtype=np.float32)
h = 120 / (1 + np.exp(-0.01 * (k - 300))) + 10 * np.cos(2 * np.pi * k / (24*7))
t = np.arange(T, dtype=np.float32)
input_series_2 = h[(t // 60).astype(int)] + 2 * np.sin(2 * np.pi * t / 60) + rng.normal(0, 0.5, size=T)

In [35]:
multi_series_forecasts = model.forecast([input_series_1, input_series_2], horizon_len=128)